In [ ]:
#default_exp loader

In [ ]:
#hide
from nbdev.showdoc import *

# Loader

The loader takes a directory or list of image files and checks them against database or checkpoint. If there is a saved checkpoint and the files haven't changed, it loads the checkpoint and sends the data directly to Ranker. If not, it sends them to Crafter. Ideally  it could send new images to Crafter and load dictionary of old images at the same time, without re-encoding old images.

> API details.

## File manipulation

In [ ]:
#export
from pathlib import Path
from PIL import Image
from tqdm import tqdm

In [ ]:
#export
def slugify(filepath):
    return f'{filepath.stem}_{str(filepath.stat().st_mtime).split(".")[0]}'

def verify_image(f):
    try:
        img = Image.open(f) # open the image file
        img.verify() # verify that it is, in fact an image
        return(True)
    except Exception as e:
        print(f'Skipping bad file: {f}\ndue to {type(e)}')
        pass
    
def get_image_files(path):
    img_extensions = ['.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp']
    return [(f, slugify(f)) for f in tqdm(path.rglob('*')) if f.suffix in img_extensions]

In [ ]:
root = Path('./images')


filepaths = get_image_files(root)

len(filepaths)

85it [00:00, 59419.31it/s]


82

In [ ]:
filepaths[:3]

[(PosixPath('images/Wholesome-Meme-3.jpg'), 'Wholesome-Meme-3_1621298477'),
 (PosixPath('images/Wholesome-Meme-44.png'), 'Wholesome-Meme-44_1621298480'),
 (PosixPath('images/Wholesome-Meme-69.jpg'), 'Wholesome-Meme-69_1621298481')]

## Loaders

So we have a list of paths and slugified filenames from the folder. We want to see if there's an archive, so that we don't have to recalculate tensors for images we've seen before. Then we want to pass that directly to the indexer, but send the new images through the crafter and encoder first.



In [ ]:
#export
import torch
import torchvision

We want to use the GPU, if possible, for all the pyTorch functions. But if we can't get access to it we need to fallback to CPU. Either way we call it `device` and pass it to each function in the executors that use torch.

In [ ]:
#export
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

The `archive_loader` is only called in `indexFlow`. It takes the list of image files and the folder they're in (and the torch device), opens an archive if there is one

In [ ]:
#export
def archive_loader(filepaths, root, device):
    dbpath = root/'memery.pt'
#     dbpath_backup = root/'memery.pt'
    db = db_loader(dbpath, device)
    
    current_slugs = [slug for path, slug in filepaths]    
    archive_db = {i:db[item[0]] for i, item in enumerate(db.items()) if item[1]['slug'] in current_slugs}      
    archive_slugs = [v['slug'] for v in archive_db.values()]
    new_files = [(str(path), slug) for path, slug in filepaths if slug not in archive_slugs and verify_image(path)]
    
    return(archive_db, new_files)

The `db_loader` takes a location and returns either the archive dictionary or an empty dictionary. Decomposed to its own function so it can be called separately from `archive_loader` or `queryFlow`. 

In [ ]:
#export
def db_loader(dbpath, device):

    # check for savefile or backup and extract
    if Path(dbpath).exists():
        db = torch.load(dbpath, device)
#     elif dbpath_backup.exists():
#         db = torch.load(dbpath_backup)
    else:
        db = {}
    return(db)

The library `annoy`, [Approximate Nearest Neighbors Oh Yeah!](https://github.com/spotify/annoy) allows us to search through vector space for approximate matches instead of exact best-similarity matches. We sacrifice accuracy for speed, so we can search through tens of thousands of images in less than a thousand times the time it would take to search through tens of images. There's got to be a better way to put that.

In [ ]:
#export
from annoy import AnnoyIndex

In [ ]:
#export
def treemap_loader(treepath):
    treemap = AnnoyIndex(512, 'angular')

    if treepath.exists():
        treemap.load(str(treepath))
    else:
        treemap = None
    return(treemap)

In [ ]:
treepath = Path('images/memery.ann')

In [ ]:
treemap = AnnoyIndex(512, 'angular')

In [ ]:
if treepath.exists():
    treemap.load(str(treepath))
else:
    treemap = None

Here we just test on the local image folder

In [ ]:
archive_db, new_files = archive_loader(get_image_files(root), root, device)

85it [00:00, 53092.46it/s]


Skipping bad file: images/corrupted-file.jpeg
due to <class 'PIL.UnidentifiedImageError'>
Skipping bad file: images/.ipynb_checkpoints/corrupted-file-checkpoint.jpeg
due to <class 'PIL.UnidentifiedImageError'>


In [ ]:
len(archive_db), len(new_files), treemap.get_n_items()

(80, 0, 80)

In [ ]:
dbpath = root/'memery.pt'
#     dbpath_backup = root/'memery.pt'
db = db_loader(dbpath, device)

current_slugs = [slug for path, slug in filepaths]    

In [ ]:
archive_db = {i:db[item[0]] for i, item in enumerate(db.items()) if item[1]['slug'] in current_slugs}  

In [ ]:
len(archive_db)

80